In [263]:
import pandas as pd
import numpy as np

telecom_users = pd.read_csv('../input/appleimage/telecom_users.csv')
telecom_users.head()

,Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1869,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No
1,4528,9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,...,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.2,No
2,6344,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes
3,6739,6994-KERXL,Male,0,No,No,4,Yes,No,DSL,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.5,No
4,432,2181-UAESM,Male,0,No,No,2,Yes,No,DSL,...,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.5,No


In [264]:
telecom_users['TotalCharges'] = pd.to_numeric(telecom_users['TotalCharges'], errors = 'coerce')

In [265]:
telecom_users['TotalCharges'].fillna(telecom_users['TotalCharges'].mean(), inplace = True)

In [266]:
telecom_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5986 entries, 0 to 5985
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        5986 non-null   int64  
 1   customerID        5986 non-null   object 
 2   gender            5986 non-null   object 
 3   SeniorCitizen     5986 non-null   int64  
 4   Partner           5986 non-null   object 
 5   Dependents        5986 non-null   object 
 6   tenure            5986 non-null   int64  
 7   PhoneService      5986 non-null   object 
 8   MultipleLines     5986 non-null   object 
 9   InternetService   5986 non-null   object 
 10  OnlineSecurity    5986 non-null   object 
 11  OnlineBackup      5986 non-null   object 
 12  DeviceProtection  5986 non-null   object 
 13  TechSupport       5986 non-null   object 
 14  StreamingTV       5986 non-null   object 
 15  StreamingMovies   5986 non-null   object 
 16  Contract          5986 non-null   object 


In [267]:
telecom_users.drop('customerID', inplace = True, axis = 1)

In [268]:
y = telecom_users['Churn']

In [269]:
X = telecom_users.drop(['Churn'], axis = 1)

In [270]:
num_features = [cols for cols in X.columns if X[cols].dtype in ['int64', 'float64']]
num_features.remove('SeniorCitizen')

In [271]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

for i in num_features:
    k = np.array(X[i])
    k = k.reshape(-1, 1)
    K = scaler.fit_transform(k)
    X[i] = K

In [272]:
cat_features = [cols for cols in X.columns if X[cols].dtype == 'object']

In [273]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

for i in cat_features:
    k = X[i]
    K = encoder.fit_transform(k)
    X[i] = K

In [274]:
Y = encoder.fit_transform(y)
y = Y

In [275]:
from collections import Counter

print(Counter(y))

Counter({0: 4399, 1: 1587})


In [276]:
X.head()

,Unnamed: 0,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,0.265408,1,0,1,1,1.000000,1,2,2,1,1,1,1,1,1,2,0,1,0.058209,0.197998
1,0.642999,0,0,0,0,0.611111,1,0,1,0,2,2,0,2,0,0,1,1,0.695522,0.456312
2,0.900880,0,1,1,0,0.527778,1,2,1,0,0,0,0,0,0,0,1,0,0.564179,0.328993
3,0.956972,1,0,0,0,0.055556,1,0,0,0,0,0,0,0,2,0,1,2,0.374627,0.025352
4,0.061346,1,0,0,0,0.027778,1,0,0,2,0,2,0,0,0,0,0,2,0.350249,0.011620


In [277]:
from imblearn.over_sampling import SMOTE

sampler = SMOTE(sampling_strategy = 0.8)
Xs, ys = sampler.fit_resample(X, y)

print(Counter(ys))

Counter({0: 4399, 1: 3519})


In [278]:
X = Xs
y = ys

In [279]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size = 0.2)

In [280]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, plot_roc_curve

model = SVC(gamma = 1, C = 5)

model.fit(X_train, y_train)

predictions = model.predict(X_test)

ac = accuracy_score(predictions, y_test)

cr = classification_report(predictions, y_test)

auc = roc_auc_score(predictions, y_test)

#0 - Not Exited, 1 - Exited

print('classification_report', '\n', cr, '\naccuracy_score :', ac, '\nroc_auc_score :', auc)

classification_report 
               precision    recall  f1-score   support

           0       0.84      0.82      0.83       907
           1       0.77      0.79      0.78       677

    accuracy                           0.81      1584
   macro avg       0.81      0.81      0.81      1584
weighted avg       0.81      0.81      0.81      1584
 
accuracy_score : 0.8099747474747475 
roc_auc_score : 0.8076612397583867


In [281]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

model2 = XGBClassifier(n_estimators = 1000)

model2.fit(X_train, y_train)

predictions = model2.predict(X_test)

ac2 = accuracy_score(predictions, y_test)

cr2 = classification_report(predictions, y_test)

auc2 = roc_auc_score(predictions, y_test)

print('classification_report', '\n', cr2, '\naccuracy_score :', ac2, '\nroc_auc_score :', auc2)

classification_report 
               precision    recall  f1-score   support

           0       0.81      0.83      0.82       867
           1       0.79      0.77      0.78       717

    accuracy                           0.80      1584
   macro avg       0.80      0.80      0.80      1584
weighted avg       0.80      0.80      0.80      1584
 
accuracy_score : 0.8036616161616161 
roc_auc_score : 0.800618204456284
